In [1]:
import os 
import glob

In [2]:
import pandas as pd
import nibabel as nb 
import numpy as np 

In [3]:
# define paths 
d="output" 
base="adc_allb_mc__wildbootstrap_iter_"
ext=".nii.gz"

In [4]:
# get files 
files = glob.glob(d+"/"+base + "*")
assert files 

In [5]:
# get segmentation (we compute CoV over one kidney) 
segf = d+"/seg.nii.gz"
assert os.path.exists(segf)

In [15]:
# get all the labels 
seg = nb.load(segf).get_fdata()
labels = np.unique(seg)
labels = [l for l in labels if l!=0]

In [16]:
# create array to hold pandas dataframes
dfs = []

In [17]:
# load files in numpy 
for label in labels:
    for i, f in enumerate(sorted(files)): 
        # load file 
        im = nb.load(f).get_fdata()
    
        # select masked area
        array = im[seg==label]

    
        # assign to pandas dataframe 
        df = pd.DataFrame({'adc':array})
        df = df.assign(label=label,iteration=i)
        dfs.append(df)
    
    
        

In [18]:
dfs = pd.concat(dfs)

In [19]:
dfs

,adc,label,iteration
0,0.002061,1.0,0
1,0.002180,1.0,0
2,0.002195,1.0,0
3,0.002152,1.0,0
4,0.002095,1.0,0
...,...,...,...
836,0.002108,1.0,11
837,0.002065,1.0,11
838,0.001936,1.0,11
839,0.001964,1.0,11


In [24]:
# let's compute CoV for each label separately 
CoV = df.groupby('label').agg(
    mean_adc=('adc', 'mean'),
    std_adc=('adc', 'std'),
    cv_adc=('adc', lambda x: 100*np.std(x, ddof=0) / np.mean(x))
).reset_index()


In [25]:
CoV 

,label,mean_adc,std_adc,cv_adc
0,1.0,0.002221,0.000325,14.638096
